# SSW2023 petitRADTRANS Intro Exercises

## Author: Paul Molliere (Max Planck Institute for Astronomy)

Run the [SWW2023_Retrievals_using_petitRADTRANS_Setup](https://colab.research.google.com/drive/1KwHnZsjScd6v1vmF1qdJN2iHkLRK-dt9?usp=sharing) notebook to download the data. The setup notebook needs to just be run **once** once for Hands-on Session IV.

## Google Colab Usage

*Please read (don't just hit run) the information given above each code cell as there are separate install cells for Colab*
&#128992;
*and running Python on your computer*
&#128309;.

**Confirm login account**
* Please make sure to be logged in with the Google account you want to use for the exercises before running the code cells below. You can check by clicking the circular account icon in the top right corner of the colab notebook.

**Working directory**
* Note: The software and data will be installed in a directory called "SSW2023/pRT" in your Google drive. This directory will be created if it does not exist.

**Running cells**
* Run cells individually by clicking on the triangle on each cell

**To Restart runtime**
*   Click on Runtime menu item
*   Select Restart runtime
*   Select Run code cells individually from the top

**To Recreate runtime**
*   Click on Runtime menu item
*   Select Disconnect and Delete runtime
*   Select Run code cells individually from the top

**To Exit:**
*   Close the browser window

# &#128992; When running on Colab: pRT installation and setup
This cell installs everything you need in Colab, as explained in the hands-on session's setup documentation.

&#128992; **Run this cell if you are running on Colab**

In [ ]:
import os

# Install MultiNest
os.chdir('/content/')
!rm -rf multinest
!git clone https://www.github.com/johannesbuchner/multinest.git
!cd multinest/build && cmake ..
!sed -i 's/-lmkl_gf_lp64 -lmkl_gnu_thread/-lmkl_mc3 -lmkl_rt -lmkl_avx2/' multinest/build/src/CMakeFiles/multinest_shared.dir/link.txt
!cd multinest/build && make && make install
!cp /content/multinest/lib/* /lib/

# Install petitRADTRANS (pRT)
import numpy as np
!pip install --no-cache-dir -U petitRADTRANS

&#128992; **Run the  2 cells if you are running on Colab**

This cell mounts the Google Drive (please allow it to do so) and specifies the path of the pRT input data folder. If you *are* running on Colab, but changed the default path specified in the setup notebook, please modify the path to correctly point to the input data in your Google Drive.

"SSW2023/pRT" is the default and you can leave that as-is or change it in the fill in box on the right. Be sure to pick a directory name that does not have any spaces. *This must match the directory used in the setup notebook.* This cell must be run to define the data location.

In [ ]:
# If you update the directory in the box on the right, re-run this cell
pRT_dir = 'SSW2023/pRT' #@param {type:"string"}

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Google top level drive dir
drive_dir = "/content/drive/MyDrive/"

# pRT directory path
pRT_path = os.path.join(drive_dir, pRT_dir)

# Confirm directory already exists per Setup
if os.path.exists(pRT_path):
  print("OK! directory '%s' exists" %pRT_path)
else:
  print("Run Setup directory '%s' does not exist" %pRT_path)

# Specify location of pRT's input data
os.environ['pRT_input_data_path'] = \
                  pRT_path+'/pRT_retrieval_SSW/input_data'

# &#128309; When running on your computer:
Please modify `absolute_path_of_the_pRT_retrieval_SSW_folder_on_your_machine` below to point to the `pRT_retrieval_SSW` folder you downloaded as described in the documentation. Then run the cell below to set the `pRT_input_data_path` environment variable accordingly. This is done such that pRT knows where the input data is.

&#128309; **Run this cell if running on your computer (not Colab!)**

In [ ]:
absolute_path_of_the_pRT_retrieval_SSW_folder_on_your_machine = '' # Please complete!

import os
import numpy as np

os.environ['pRT_input_data_path'] = \
    os.path.join(absolute_path_of_the_pRT_retrieval_SSW_folder_on_your_machine,
                 'input_data')

# petitRADTRANS: introduction exercises
## Defining an atmosphere and calculating its emission spectrum
To get started, let's calculate the emission spectrum of a hot Jupiter. The detailed description of everything that pRT allows you to do can be found in the [online tutorial](https://petitradtrans.readthedocs.io/en/latest/content/tutorial.html). Because we don't have too much time, we will only discuss a subset of this.

We start by making a pRT object, which we call `atmosphere` here; we tell it which opacity species to use, and over which wavelength range. More information on which opacity species are available can be found [here](https://petitradtrans.readthedocs.io/en/latest/content/available_opacities.html), but note that we only have a smaller selection available for the condensed input data folder we use for the Sagan Workshop. Flags like `_HITEMP` reference a specific opacity data base, `R_200` means that the opacity resolution ($\lambda / \Delta \lambda$) is equal to 200 in our example. [pRT allows users to bin down line opacities to any resolution](https://petitradtrans.readthedocs.io/en/latest/content/notebooks/Rebinning_opacities.html).

In [ ]:
import os
import numpy as np
from petitRADTRANS import Radtrans

# Line absorbers:
line_species = ['H2O_Exomol_R_200', # _Exomol stands for the water line list from Exomol.
                'CO_all_iso_HITEMP_R_200', # _R_200 means that the opacity resolution is 200.
                'CH4_hargreaves_R_200',
                'CO2_R_200',
                'SO2_R_200']

# Rayleigh scatterers
rayleigh_species = ['H2', 'He']

# Collision-induced absorption opacity
continuum_opacities = ['H2-H2', 'H2-He']

# Wavelength range for calculating spectra, in micrometer
wlen_bords_micron = [0.3, 15]

atmosphere = Radtrans(line_species = line_species,
                      rayleigh_species = rayleigh_species,
                      continuum_opacities = continuum_opacities,
                      wlen_bords_micron = wlen_bords_micron)

Next we define the vertical structure of the atmosphere, we will use 100 layers, equidistantly spaced in log(pressure) from 1 $\mu$bar to 100 bar.

In [ ]:
pressures = np.logspace(-6, 2, 100)
# tell the pRT object about the pressure grid to be used
atmosphere.setup_opa_structure(pressures)

Next we define the atmospheric temperature structure, using a [Guillot (2010)](https://ui.adsabs.harvard.edu/abs/2010A%26A...520A..27G/abstract) temperature profile. It has the following free parameters:
- $\kappa_{\rm IR}$ is the average infrared opacity (so photon absorption cross-section per unit mass of atmospheric material).
- $\gamma$ is the ratio between the optical and infrared opacity and controls whether temperature inversions form in the atmosphere.
- $T_{\rm int}$ is the internal temperature of the planet, defined such that the net planet flux radiated to space at the top of the atmosphere is $F_{\rm int}=\sigma T_{\rm int}^4$, where $\sigma$ is the Stefan-Boltzmann constant.
- $T_{\rm equ}$ is the equilibrium temperature of the planet, expressing the strength of irradiation, defined such that the planet-wide average (over the full sphere) of the stellar flux received at the location of the planet is $F_{\rm equ}=\sigma T_{\rm equ}^4$. It holds that $T_{\rm eff}^4 = T_{\rm equ}^4 + T_{\rm int}^4$, where $T_{\rm eff}$ is the planet's effective temperature.

The temperature profile is then defined like this:
$$T(\tau)^4 = \frac{3T_{\rm int}^4}{4}\left(\frac{2}{3}+\tau\right) + \frac{3T_{\rm equ}^4}{4}\left[\frac{2}{3}+\frac{1}{\gamma\sqrt{3}}+\left(\frac{\gamma}{\sqrt{3}}-\frac{1}{\gamma\sqrt{3}}\right)e^{-\gamma\tau\sqrt{3}}\right].$$
Here, $\tau$ is the so-called optical depth, which is defined as altitude in units of the mean free path of a photon. $\tau = 0$ denotes the top of the atmosphere and we go deeper (to lower altitudes) into the atmosphere for $\tau > 0$. So, when moving from the top to an altitude corresponding to $\tau=3$, say, a photon would on average be absorbed three times (which makes it unkikely, but not impossible, that a photon travels that far). One can approximate $\tau = P\kappa_{\rm IR}/g$, where $g$ is the gravitational acceleration in the atmosphere and $P$ is the pressure. In this way we can transform from $T(\tau)$ to $T(P)$.

**Please note that the Guillot (2010) formulation is "only" a useful mathematical approximation of how atmospheric temperature structures behave upon variation of fundamental input parameters. It is a useful qualitative tool, but cannot predict atmospheric pressure-temperature structures quantitatively. In general things are more complicated. Atmospheres do not just have one infrared opacity and one $\gamma$ value, instead both vary as function of wavelength and altitude. In addition, Guillot (2010) assumes that the atmosphere is 1-d, that all energy is transported by radiation, etc.** The Guillot formulation may still be a useful function to parameterize the shape of atmospheric temperature structures in a retrieval, however. In this case the best-fit parameters would not tell you anything about the true values of $\kappa_{\rm IR}$, $\gamma$, etc., since a retrieval does not know anything about the physical interpretation we associate with parameters. In this case the Guillot parameters are merely a knob to tune the temperature profile to the right shape.

In [ ]:
import pylab as plt
plt.rcParams['figure.figsize'] = (10, 6)

from petitRADTRANS.physics import guillot_global

# All I/O in petitRADTRANS in in cgs units,
# other than pressure (which is in bars) and and MMW (which is in units of amu)
# Internally, pRT does everything in cgs units.

gravity = 10**3.5 # Typical hot Jupiter gravity value

kappa_IR = 0.01 # Typical hot Jupiter opacity value.
gamma = 0.4 # This gamma leads to a non-inverted atmosphere.
T_int = 200. # Internal temperature, set by the cooling rate of the planet.
T_equ = 1500. # Euqilibrium temperature, set by the irradiation strength of the star.

temperature = guillot_global(pressures, kappa_IR, gamma, gravity, T_int, T_equ)

This is what it looks like:

In [ ]:
plt.plot(temperature, pressures)
plt.yscale('log')
plt.xlim([1300, 1750])
plt.ylim([1e2, 1e-6])
plt.xlabel('T (K)')
plt.ylabel('P (bar)')
plt.show()

### Exercise 1:
Currently $\gamma=0.4$, so the optical opacity absorbing the stellar light is smaller than the infrared opacity. By filling in the blanks below, Change $\gamma$ to 2.5 and save the resulting temperature structure in an array called `inverted_temperature`. Plot `temperature` and `inverted_temperature`. How do they compare? What do you think is happening? Please also write your answer to the questions in the empty text field below the code cell. [You can click here to see if your solution is correct](https://www2.mpia-hd.mpg.de/~molliere/SSW/solutions_notebook_1.html#Exercise-1:).

In [ ]:
gamma = ... # Fill in
inverted_temperature = ... # Fill in

plt.plot(temperature, pressures, label = '$\gamma = 0.4$')
plt.plot(inverted_temperature, pressures, label = '$\gamma = 2.5$')
plt.yscale('log')
plt.xlim([1300, 1750])
plt.ylim([1e2, 1e-6])
plt.xlabel('T (K)')
plt.ylabel('P (bar)')
plt.legend()
plt.show()

Now we define the atmospheric composition by providing mass fractions of absorbing species in every layer. We multiply the values by `np.ones_like(temperature)`, because they are defined layer-by-layer; here we chose a vertically constant prescription. But you can also use [chemical equilibrium abundances in pRT](https://petitradtrans.readthedocs.io/en/latest/content/notebooks/poor_man.html), for example.

In [ ]:
mass_fractions = {}
# Typical values for H2/He dominated atmospheres
mass_fractions['H2'] = 0.74 * np.ones_like(temperature)
mass_fractions['He'] = 0.24 * np.ones_like(temperature)
mass_fractions['H2O'] = 0.001 * np.ones_like(temperature)
mass_fractions['CO'] = 0.005 * np.ones_like(temperature)
mass_fractions['CO2'] = 0.00001 * np.ones_like(temperature)
mass_fractions['SO2'] = 0.00001 * np.ones_like(temperature)
mass_fractions['CH4'] = 0.000001 * np.ones_like(temperature)

# Mean molecular weight in units of amu
MMW = 2.33 * np.ones_like(temperature) # 2.3 is the typical value for a H2/He dominated atmosphere

If we wanted to be fully correct, we'd make sure that the mass fractions sum up to 1 in every layer. This is especially important if we want to calculate the mean molecular weight (MMW) from the atmospheric composition. In fact we will enforce this in the retrievals studied in the second exercise notebook. Since we provide a MMW explictly here, and want to simply study what a typical hot Jupiter spectrum looks like, it is enough to just specify order-of-magnitude correct values for all atmospheric absorbers.

Finally, **let's calculate the flux emitted by the exoplanet atmosphere!**

In [ ]:
atmosphere.calc_flux(temperature, mass_fractions, gravity, MMW)

And now we plot it. Note the use of `nc.c` to convert from frequency to wavelength. `nc.c` is the speed of light in cm/s as defined in pRT's `n`atural `c`onstant package `nat_cst`. It is imported like this: `from petitRADTRANS import nat_cst as nc`. See [here](https://petitradtrans.readthedocs.io/en/latest/content/nat_cst_doc.html) for a list of all pre-defined natural constants (all are defined in cgs units).


In [ ]:
from petitRADTRANS import nat_cst as nc

plt.plot(nc.c/atmosphere.freq/1e-4, atmosphere.flux/1e-6)

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'Planet flux $F_\nu$ (10$^{-6}$ erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$)')
plt.show()

### Exercise 2:
First we save `atmosphere.flux` to a variable called `flux_non_inverted`. Then, filling in the blanks, repeat the flux caclulation using the inverted temperature structure from above (`inverted_temperature`). Show both fluxes in one plot. What can you see? Why does this happen? Please also write your answer to the questions in the empty text field below the code cell. [You can click here to see if your solution is correct](https://www2.mpia-hd.mpg.de/~molliere/SSW/solutions_notebook_1.html#Exercise-2:).

In [ ]:
atmosphere.calc_flux(temperature, mass_fractions, gravity, MMW)
flux_non_inverted = atmosphere.flux
atmosphere.calc_flux(..., mass_fractions, gravity, MMW) # Fill in
flux_inverted = ... # Fill in

plt.plot(nc.c/atmosphere.freq/1e-4, flux_non_inverted/1e-6, label = 'No temperature inversion')
plt.plot(nc.c/atmosphere.freq/1e-4, flux_inverted/1e-6, label = 'Temperature inversion')

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'Planet flux $F_\nu$ (10$^{-6}$ erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$)')
plt.legend()
plt.show()

### Exercise 3:
By filling in the blanks, set up the atmosphere to be isothermal, at 1500 K (so replace the temperature array in `calc_flux()` with an array containing only the value `1500.`). Compare the resulting spectrum with the inverted and non-inverted emission spectra from above. What do you observe, and why? Please also write your answer to the questions in the empty text field below the code cell. [You can click here to see if your solution is correct](https://www2.mpia-hd.mpg.de/~molliere/SSW/solutions_notebook_1.html#Exercise-3:).

In [ ]:
atmosphere.calc_flux(..., mass_fractions, gravity, MMW) # Fill in
flux_isothermal = ... # Fill in

plt.plot(nc.c/atmosphere.freq/1e-4, flux_non_inverted/1e-6, label = 'No temperature inversion')
plt.plot(nc.c/atmosphere.freq/1e-4, flux_inverted/1e-6, label = 'Temperature inversion')
plt.plot(nc.c/atmosphere.freq/1e-4, flux_isothermal/1e-6, label = 'Isothermal atmosphere')

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'Planet flux $F_\nu$ (10$^{-6}$ erg cm$^{-2}$ s$^{-1}$ Hz$^{-1}$)')
plt.legend()
plt.show()

Emission spectra of transiting planets are usually plotted as contrast, which means that the planet flux $F_{\rm P}$ is divided by the stellar flux $F_{\rm *}$. The planetary flux returned by pRT is the emergent flux at the top of the planet's atmosphere. So to get the contrast at the location of the observer we have to calculate the following:
\begin{equation}
C = \left(F_{\rm P} \cdot \frac{4\pi R_{\rm P}^2}{4\pi  d^2}\right) \left(F_{\rm *} \cdot \frac{4\pi  R_{*}^2}{4\pi  d^2}\right)^{-1} = \frac{F_{\rm P}}{F_{\rm *}} \left(\frac{R_{\rm P}}{R_{*}}\right)^2,
\end{equation}
where $d$ is the distance to the observer. **Please note that the contrast is often simply written as $F_{\rm P}/F_{\rm *}$, assuming that "F" means flux measured at the location of the observer, so not at the top of the atmosphere of the planet or star.** [pRT has a library of stellar spectra available](https://petitradtrans.readthedocs.io/en/latest/content/notebooks/nat_cst_utility.html#PHOENIX-and-ATLAS9-stellar-model-spectra). We can use this function to extract the flux at the same wavelength spacing as the pRT planet emission model:

In [ ]:
def get_stellar_spectrum(pRT_object, stellar_temperature):

    from petitRADTRANS.retrieval import rebin_give_width as rgw

    stellar_spec = nc.get_PHOENIX_spec(stellar_temperature)
    wlen_in_micron = stellar_spec[:,0]/1e-4
    flux_star = stellar_spec[:,1]

    bins_borders_take_freq = pRT_object.border_freqs
    bins_borders_take = nc.c/pRT_object.border_freqs/1e-4

    wavelens = np.array((bins_borders_take_freq[1:]+bins_borders_take_freq[:-1])/2.)
    wavelens = nc.c/wavelens/1e-4
    widths = np.diff(bins_borders_take)

    bin_flux = rgw.rebin_give_width(wlen_in_micron,
                                    flux_star,
                                    wavelens,
                                    widths)

    return bin_flux

Now, assuming that the planet has a radius of 1.5 Jupiter radii (we use `1.5*nc.r_jup`) and that the star has a radius of one solar radius (we use `nc.r_sun`) and an effective temperature of 5700 K, we plot the contrast between planet and star in units of parts-per-million (ppm):

In [ ]:
stellar_flux_at_top_of_atmosphere = get_stellar_spectrum(atmosphere, 5700.)

scaling_factor = (1.5*nc.r_jup)**2./nc.r_sun**2./stellar_flux_at_top_of_atmosphere*1e6

plt.plot(nc.c/atmosphere.freq/1e-4, flux_non_inverted*scaling_factor, label = 'No temperature inversion')
plt.plot(nc.c/atmosphere.freq/1e-4, flux_inverted*scaling_factor, label = 'Temperature inversion')
plt.plot(nc.c/atmosphere.freq/1e-4, flux_isothermal*scaling_factor, label = 'Isothermal atmosphere')

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'$F_{\rm P}/F_{\rm *}$ (ppm)')
plt.legend()
plt.show()

## Calculating transmission spectra

Above we calculated the emission spectrum of the planet with `calc_flux()`. The transmission radius of the planet (so the effective size of the planet when occulting the stellar disk) can be calculated via `calc_transm()`.

The parameteres are similar to those of `calc_flux()`, but we now additionally need to define a planetary reference radius `R_pl` and reference pressure `P_0_bar`. This is because the vertical coordinate in pRT is pressure $P$, but we have to determine the planet's radius as a function of pressure, $r(P)$, for the calculation of the transmission spectrum. This is done inside pRT by integrating the equation of hydrostatic equilibrium $dP/dr = -\rho g(r)$, where $g$ is the gravitational acceleration and $\rho$ the mass density. The reference pressure $P_0$ and reference radius $R_{\rm Pl}$ set the integration constant of the $r(P)$ solution, such that $r(P_0)=R_{\rm Pl}$.
The parameter defined as `gravity` when calling `calc_transm()` defines the gravity at the reference pressure and reference radius. In a retrieval one fixes either the reference pressure and retrieves the reference radius, or vice versa.

After calculating the transmission radius, the transit depth can be obtained by calculating $[R_{\rm P}(\lambda)/R_*]^2$, where $R_{\rm P}(\lambda)$ is the wavelength-dependent transmission radius of the planet. In the example below we again assume that the star has a radius equal to that of the sun.

In [ ]:
R_pl = 1.5*nc.r_jup
R_star = nc.r_sun

atmosphere.calc_transm(temperature,
                       mass_fractions,
                       gravity,
                       MMW,
                       R_pl=R_pl, # reference radius
                       P0_bar=0.01) # reference pressure in bar

# Extract the transit depth
transit_depth = (atmosphere.transm_rad/R_star)**2.

plt.plot(nc.c/atmosphere.freq/1e-4, transit_depth*1e6)

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'Transit depth (ppm)')
plt.show()

Now, lets add some clouds. pRT allows users to chose between many different cloud implementations which are described on the [documentation website](https://petitradtrans.readthedocs.io/en/latest/content/notebooks/clouds.html). Here we will just look at what happens if you specify a cloud (top) pressure. What pRT then does is to assume that the atmosphere is fully opaque at pressures larger than the specified cloud pressure. Since larger pressures mean lower altitudes this means that the atmosphere is fully opaque below a certain altitude.

### Exercise 4:
When calculating the transmission spectrum with `calc_transm()`, hand the function a keyword argument called `Pcloud`, which defines the cloud top pressure in bar. How does the spectrum compare to the cloud-free one if you set `Pcloud = 0.01`? Why does it look different? Please also write your answer to the questions in the empty text field below the code cell. [You can click here to see if your solution is correct](https://www2.mpia-hd.mpg.de/~molliere/SSW/solutions_notebook_1.html#Exercise-4:).

In [ ]:
atmosphere.calc_transm(temperature,
                       mass_fractions,
                       gravity,
                       MMW,
                       R_pl=R_pl, # reference radius
                       P0_bar=0.01, # reference pressure in bar
                       Pcloud = ...) # Fill in
                       # cloud top pressure in bar

transit_depth_cloudy = ... # Fill in

# Plot the results
plt.plot(nc.c/atmosphere.freq/1e-4, transit_depth*1e6, label = 'Cloud-free')
plt.plot(nc.c/atmosphere.freq/1e-4, transit_depth_cloudy*1e6, label = 'Cloudy')

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'Transit depth (ppm)')
plt.legend()
plt.show()

### Exercise 5:
Below we calculate and plot two transmission spectra: the first is identical to the cloudy spectrum above. For the second, we apply the following changes:
- We increase the mass fraction of all atmospheric species (except for H$_2$ and He) by a factor 10.
- We decrease the cloud deck pressure by a factor of 10.
- We divide the reference pressure by 9.4 (so almost a factor of 10).

How do the spectra compare? What do you think is happening? Imagine you were to retrieve the absorber mass fractions and cloud deck pressure from observations over a narrow wavelength range (e.g., 1.1 to 1.6 µm): would this be possible? Note that the 1.1 to 1.6 µm region corresponds to the wavelength range of the often-used WFC3 instrument of the Hubble Space Telescope. Please write your answer to the questions in the empty text field below the code cell. [You can click here to see if your solution is correct](https://www2.mpia-hd.mpg.de/~molliere/SSW/solutions_notebook_1.html#Exercise-5:).

In [ ]:
mass_fraction_more_metals = mass_fractions.copy()
for name in mass_fraction_more_metals.keys():
    if name not in ['H2', 'He']:
        mass_fraction_more_metals[name] = 10. * mass_fraction_more_metals[name]

atmosphere.calc_transm(temperature,
                       mass_fraction_more_metals,
                       gravity,
                       MMW,
                       R_pl=R_pl, # reference radius
                       P0_bar=0.01/9.4, # reference pressure in bar
                       Pcloud = 0.001) # cloud top pressure in bar

transit_depth_cloudier = (atmosphere.transm_rad/R_star)**2.

plt.plot(nc.c/atmosphere.freq/1e-4, transit_depth_cloudy*1e6, label = 'Cloudy')
plt.plot(nc.c/atmosphere.freq/1e-4, transit_depth_cloudier*1e6, label = 'Modified cloud model')

plt.fill_between(np.linspace(1.1, 1.6, 2), 23750, y2=24000, color = 'gray',
                 label = 'HST WFC3 wavelength range', alpha = 0.4, edgecolor = None)
plt.ylim([23750, 24000])

plt.xscale('log')
plt.xlabel('Wavelength (microns)')
plt.ylabel(r'Transit depth (ppm)')
plt.legend()
plt.show()